In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact
from scipy.stats import norm
from collections import Counter
from scipy.stats import binomtest
import os

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

def parse_table(i):
    return [int(x) for x in i.replace("[", "").replace("]", "").split(",")]

d_abrev = {"LiangSteinNeuron":"FC exc. neur.", "FetalChondrocytes":"F chond.", "SertoliMale":"FG sertoli", "preGC_IIaFemale":"FG preGC IIa",\
          "NeuralFemale":"FG neur.", "FetalGonadImmuneFemale":"FG immune", "VIP":"AC VIP inh. neur.", "LiangSteinProgenitor":"FC prog.",\
          "AdultHeartVentricularCardiomyocyte":"AH cardiomyo.", "AdultLoopOfHenle":"AK loop of henle", "FetalBrainNeurGlioblast_CB_VZ":"FCB glioblast",\
         "AdultProximalTubule":"AK prox. tub.", "FetalLeydigMale":"FG leydig", "SST":"AC SST inh neur.", "KosoyRoussosControlMicroglia":"AC microglia",\
         "FetalBrainFloorPlate":"FB fl. plate", "FetalArterialECs":"FH endoth.", "ASCT":"AC astro.", "FetalBrainCOP":"FB COP",\
         "AMY":"AA neur.", "PVALB":"AC PVALB inh neur.", "ITL23":"AC L2-3 IT neur.", "FetalBrainNeurCB_GNP_IPC_1":"FB inter. prog.", "FetalBrainNeurDAergic":"FB DA neur.",\
          "OGC":"AC Oligo.", "D1Pu":"AP D1 inh neur.", "FetalBrainNeurSerotonergic":"FB 5-HT neur.", "FetalBrainNeurDRG_2":"FS DRG neur.",\
          "FetalHeartPericytes":"FH peri.", "FetalHeartEndocardium":"FH endocard.", "FetalHeartCardiacFibroblasts":"FH fibro.", "FetalBrainNeurPurkinje_6":"FCB Purk. inh neur.",\
          "AdultHeartSmoothMuscle":"AH smooth musc.", "FetalBrainRoofPlate":"FB ro. plate"}

In [ ]:
#Make dictionaries to help plot things
ct = "LiangSteinNeuron"
v = pd.read_csv("StratifyCTS_New/" + ct + "/StratifyCTS_FilterNewTEs_Tau_Group1_SpecSup0_" + ct + ".txt.gz", sep = "\t")

d_bin_g1_all = {}
for i in np.unique(v["CTS_Cutoff"]):
    if "< " in i:
        d_bin_g1_all[i] = "< " + str(np.round(float(i.replace("< ", "")), 3))
    elif "-" in i:
        d_bin_g1_all[i] = str(np.round(float(i.split("-")[0]), 3)) + "-" + str(np.round(float(i.split("-")[1]), 3))
    elif "> " in i:
        d_bin_g1_all[i] = "> " + str(np.round(float(i.replace("> ", "")), 3))

ct = "preGC_IIaFemale"
v = pd.read_csv("StratifyCTS_New/" + ct + "/StratifyCTS_FilterNewTEs_Tau_Group2_SpecSup0_" + ct + ".txt.gz", sep = "\t")

d_bin_g2_all = {}
for i in np.unique(v["CTS_Cutoff"]):
    if "< " in i:
        d_bin_g2_all[i] = "< " + str(np.round(float(i.replace("< ", "")), 3))
    elif "-" in i:
        d_bin_g2_all[i] = str(np.round(float(i.split("-")[0]), 3)) + "-" + str(np.round(float(i.split("-")[1]), 3))
    elif "> " in i:
        d_bin_g2_all[i] = "> " + str(np.round(float(i.replace("> ", "")), 3))

ct = "LiangSteinNeuron"
v = pd.read_csv("StratifyCTS_New/" + ct + "/StratifyCTS_FilterNewTEs_RemTopCT_Tau_Group1_SpecSup0_" + ct + ".txt.gz", sep = "\t")
      
d_bin_g1_remTop = {}
for i in np.unique(v["CTS_Cutoff"]):
    if "< " in i:
        d_bin_g1_remTop[i] = "< " + str(np.round(float(i.replace("< ", "")), 3))
    elif "-" in i:
        d_bin_g1_remTop[i] = str(np.round(float(i.split("-")[0]), 3)) + "-" + str(np.round(float(i.split("-")[1]), 3))
    elif "> " in i:
        d_bin_g1_remTop[i] = "> " + str(np.round(float(i.replace("> ", "")), 3))

ct = "preGC_IIaFemale"
v = pd.read_csv("StratifyCTS_New/" + ct + "/StratifyCTS_FilterNewTEs_RemTopCT_Tau_Group2_SpecSup0_" + ct + ".txt.gz", sep = "\t")

d_bin_g2_remTop = {}
for i in np.unique(v["CTS_Cutoff"]):
    if "< " in i:
        d_bin_g2_remTop[i] = "< " + str(np.round(float(i.replace("< ", "")), 3))
    elif "-" in i:
        d_bin_g2_remTop[i] = str(np.round(float(i.split("-")[0]), 3)) + "-" + str(np.round(float(i.split("-")[1]), 3))
    elif "> " in i:
        d_bin_g2_remTop[i] = "> " + str(np.round(float(i.replace("> ", "")), 3))

ct = "LiangSteinNeuron"
v = pd.read_csv("StratifyCTS_New/" + ct + "/StratifyCTS_FilterNewTEs_RemTopTwoCT_Tau_Group1_SpecSup0_" + ct + ".txt.gz", sep = "\t")
      
d_bin_g1_remTop2 = {}
for i in np.unique(v["CTS_Cutoff"]):
    if "< " in i:
        d_bin_g1_remTop2[i] = "< " + str(np.round(float(i.replace("< ", "")), 3))
    elif "-" in i:
        d_bin_g1_remTop2[i] = str(np.round(float(i.split("-")[0]), 3)) + "-" + str(np.round(float(i.split("-")[1]), 3))
    elif "> " in i:
        d_bin_g1_remTop2[i] = "> " + str(np.round(float(i.replace("> ", "")), 3))

ct = "preGC_IIaFemale"
v = pd.read_csv("StratifyCTS_New/" + ct + "/StratifyCTS_FilterNewTEs_RemTopTwoCT_Tau_Group2_SpecSup0_" + ct + ".txt.gz", sep = "\t")

d_bin_g2_remTop2 = {}
for i in np.unique(v["CTS_Cutoff"]):
    if "< " in i:
        d_bin_g2_remTop2[i] = "< " + str(np.round(float(i.replace("< ", "")), 3))
    elif "-" in i:
        d_bin_g2_remTop2[i] = str(np.round(float(i.split("-")[0]), 3)) + "-" + str(np.round(float(i.split("-")[1]), 3))
    elif "> " in i:
        d_bin_g2_remTop2[i] = "> " + str(np.round(float(i.replace("> ", "")), 3))

"""ct = "LiangSteinNeuron"
v = pd.read_csv("StratifyCTS_New/" + ct + "/StratifyCTS_FilterNewTEs_RemTopThreeCT_Tau_Group1_SpecSup0_" + ct + ".txt.gz", sep = "\t")        

d_bin_g1_remTop3 = {}
for i in np.unique(v["CTS_Cutoff"]):
    if "< " in i:
        d_bin_g1_remTop3[i] = "< " + str(np.round(float(i.replace("< ", "")), 3))
    elif "-" in i:
        d_bin_g1_remTop3[i] = str(np.round(float(i.split("-")[0]), 3)) + "-" + str(np.round(float(i.split("-")[1]), 3))
    elif "> " in i:
        d_bin_g1_remTop3[i] = "> " + str(np.round(float(i.replace("> ", "")), 3))

ct = "preGC_IIaFemale"
v = pd.read_csv("StratifyCTS_New/" + ct + "/StratifyCTS_FilterNewTEs_RemTopThreeCT_Tau_Group2_SpecSup0_" + ct + ".txt.gz", sep = "\t")

d_bin_g2_remTop3 = {}
for i in np.unique(v["CTS_Cutoff"]):
    if "< " in i:
        d_bin_g2_remTop3[i] = "< " + str(np.round(float(i.replace("< ", "")), 3))
    elif "-" in i:
        d_bin_g2_remTop3[i] = str(np.round(float(i.split("-")[0]), 3)) + "-" + str(np.round(float(i.split("-")[1]), 3))
    elif "> " in i:
        d_bin_g2_remTop3[i] = "> " + str(np.round(float(i.replace("> ", "")), 3))"""


In [ ]:
#Tool to round cutoffs to make them easier to plot
def convert_cutoff(x):
    x = list(np.unique(x))
    x = [i.replace(" ", "").replace("<", "").replace(">", "").replace("=", "") for i in x]
    x_new = []
    for i in x:
        if "-" in i:
            x_new.append(float(i.split("-")[0]))
        else:
            x_new.append(float(i))
    x_new.sort()
    new = []
    c = 0
    for i in x_new:
        c+=1
        new.append(c)
    return x_new, new


In [ ]:
#Collect all the results across cell types
group1 = ["KosoyRoussosControlMicroglia", "LiangSteinNeuron", "AdultHeartVentricularCardiomyocyte", "AdultProximalTubule", "FetalArterialECs", "FetalChondrocytes", "SertoliMale", "ASCT"]
group2 = ["FetalGonadImmuneFemale", "ITL23", "AdultHeartSmoothMuscle", "AdultLoopOfHenle", "FetalHeartPericytes", "FetalHeartCardiacFibroblasts", "preGC_IIaFemale", "FetalBrainCOP"]

df = pd.DataFrame()

cutoff = 0.6
for folder in os.listdir("StratifyCTS_New"):
    print(folder)

    if folder in group1:
        vr = pd.read_csv("StratifyCTS_New/" + folder + "/StratifyCTS_FilterNewTEs_RemTopTwoCT_Tau_Group1_SpecSup0_" + folder + ".txt.gz", sep = "\t")
    #if folder in group2:
    #    vr = pd.read_csv("StratifyCTS_New/" + folder + "/StratifyCTS_FilterNewTEs_RemTopTwoCT_Tau_Group2_SpecSup0_" + folder + ".txt.gz", sep = "\t")
    
        v2r = vr[vr["Proportion"] == cutoff]
        v2r = v2r[v2r["EffectOrTotal"] == "Mean_CTS_Effect"]
        v2r = v2r[v2r["Metric"] == "Absolute log fold-change"]
        v2r = v2r[v2r["Top_20th"] == "All"]
        v2r["CutNum"] = v2r["CutNum"].astype(str)
        v2r["CTS_Rank"] = np.repeat(convert_cutoff(v2r["CTS_Cutoff"])[1], v2r.shape[0]/8)
        #v2r = v2r[v2r["CutNum"].isin(["50000", "Rest"])]
        v2r["Cell type"] = np.repeat(d_abrev[folder], v2r.shape[0])
        v2r
        df = pd.concat([df, v2r])
df

In [ ]:
#Make plot
new = []
for index, row in df.iterrows():
    new.append(d_bin_g1_remTop2[row["CTS_Cutoff"]])
#df["Tau bin"] = df["CTS_Cutoff"]
df["Tau bin"] = new

medians = []
ci = []
palette = {}
for ct in np.unique(df["Cell type"]):
    for binn in np.unique(df["CTS_Rank"]):
        dff = df[(df["Cell type"].isin([ct])) & (df["CTS_Rank"].isin([binn]))]
        dff = dff.sort_values("alpha")
        ci.append([ct, binn, list(dff["alpha"])[25], list(dff["alpha"])[975]])
        if binn == 8:
            medians.append([ct, binn, np.median(dff["alpha"]) - max(save_bin, 0)])
        if binn == 1:
            save_bin = np.median(dff["alpha"])
    
df_med = pd.DataFrame(medians).sort_values(2, ascending = False)
order = list(df_med[0])

df_ci = pd.DataFrame(ci)
df_ci.columns = ["CellType", "Bin", "Lower95", "Upper95"]
df_ci = df_ci.sort_values("Lower95", ascending = False)

#for index, row in df_ci.iterrows():
#    if row["Lower95"] > 0:
#        palette[row["CellType"]] = "#F42FF5"
#    else:
#        palette[row["CellType"]] = "#40A94D"

fig, ax = plt.subplots(figsize = (12, 6))
sns.boxplot(data = df, y = "alpha", x = "Cell type", order = order, hue = "Tau bin", palette = "rocket")
plt.ylim(-0.125, 0.1)
plt.ylabel("$\\alpha_{CA}$", size = 18)
plt.xticks(rotation = 90, size = 16)
plt.xlabel("Cell type", size = 18)
plt.xticks(size = 14)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("Removing FC exc. neur. and AC microglia", size = 21)
#plt.title("$\\alpha_{CA}$ across different levels of cell type-specificity, group 2", size = 21)
df_ci

In [ ]:
#Collect use CTS results
group1 = ["KosoyRoussosControlMicroglia", "LiangSteinNeuron", "AdultHeartVentricularCardiomyocyte", "AdultProximalTubule", "FetalArterialECs", "FetalChondrocytes", "SertoliMale", "ASCT"]
group2 = ["FetalGonadImmuneFemale", "ITL23", "AdultHeartSmoothMuscle", "AdultLoopOfHenle", "FetalHeartPericytes", "FetalHeartCardiacFibroblasts", "preGC_IIaFemale", "FetalBrainCOP"]

df = pd.DataFrame()

cutoff = 0.8
for folder in os.listdir("UseCTS_Results"):
    print(folder)

    if folder in group1:
        vr = pd.read_csv("UseCTS_Results_New/" + folder + "/UseCTS_FilterNewTEs_Tau_Group1_SpecSup0_abs_logfc_" + folder + ".txt.gz", sep = "\t")
    #if folder in group2:
    #    vr = pd.read_csv("UseCTS_Results_New/" + folder + "/UseCTS_FilterNewTEs_Tau_Group1_SpecSup0_abs_logfc_" + folder + ".txt.gz", sep = "\t")
    
        v2r = vr[vr["Proportion"] == cutoff]
        #v2r = v2r[v2r["Metric"] == "Mean_CTS_Effect"]
        v2r = v2r[v2r["Top_20th"] == "All"]
        
        keep = ""
        for i in np.unique(v2r["LFC_Cutoff"]):
            if ">" in i:
                keep = i
        v2r = v2r[v2r["LFC_Cutoff"] == keep]
        v2r["Cell type"] = np.repeat(d_abrev[folder], v2r.shape[0])
        df = pd.concat([df, v2r])
df

In [ ]:
#Replace names
df["Metric"] = df["Metric"].replace("Mean_CTS_Effect", r"$\alpha_{CTS}$")
df["Metric"] = df["Metric"].replace("Absolute log fold-change", r"$\alpha_{CA}$")

In [ ]:
#Make plots

medians = []
ci = []
palette = {}
for ct in np.unique(df["Cell type"]):
    dff = df[(df["Cell type"].isin([ct]))]
    dff = dff.sort_values("alpha")
    dff2 = dff[dff["Metric"] == r"$\alpha_{CTS}$"]
    ci.append([ct, list(dff2["alpha"])[25], list(dff2["alpha"])[975], "CTS"])
    medians.append([ct, np.median(dff2["alpha"]), "CTS"])
    dff2 = dff[dff["Metric"] == r"$\alpha_{CA}$"]
    ci.append([ct, list(dff2["alpha"])[25], list(dff2["alpha"])[975], "CA"])
    medians.append([ct, np.median(dff2["alpha"]), "CA"])


df_med = pd.DataFrame(medians)
df_med_cts = df_med[df_med[2] == "CTS"]
df_med_cts.columns = [0, "Median CTS", "CTS"]
df_med_ca = df_med[df_med[2] == "CA"]
df_med_ca.columns = [0, "Median CA", "CA"]

df_med = df_med_cts.set_index(0).join(df_med_ca.set_index(0))
df_med["Dif"] = df_med["Median CTS"] - df_med["Median CA"]
df_med = df_med.sort_values("Dif", ascending = False)
order = list(df_med.index)

df_ci = pd.DataFrame(ci)
df_ci.columns = ["CellType", "Lower95", "Upper95", "Type"]
df_ci = df_ci.sort_values("Lower95", ascending = False)

for index, row in df_ci.iterrows():
    if row["Lower95"] > 0:
        palette[row["CellType"]] = "#F42FF5"
    else:
        palette[row["CellType"]] = "#40A94D"

fig, ax = plt.subplots(figsize = (13, 6))
sns.boxplot(data = df, y = "alpha", x = "Cell type", order = order, hue = "Metric", palette = {r"$\alpha_{CTS}$":'#B864DB', r"$\alpha_{CA}$":'#DB4948'})
plt.ylim(-0.025, 0.1)
plt.ylabel("$\\alpha$", size = 26)
plt.xticks(rotation = 45, size = 16, horizontalalignment='right')
plt.xlabel("Cell type", size = 22)

plt.legend(bbox_to_anchor=(1, 1), fontsize = 22)
plt.title("Positive selection on sites with cell type-specific effects", size = 26)

df_ci

In [ ]:
#Read in the data
v = pd.read_csv("Fixed_LiangSteinNeuron.txt.gz", sep = "\t")
vv = pd.read_csv("Poly_MAF0.25_LiangSteinNeuron.txt.gz", sep = "\t")


try:
    vv = add_unfold(vv)
    vv_ref = vv[vv["Human ref"] == vv["Chimp ref"]]
    vv_alt = vv[vv["Human alt"] == vv["Chimp ref"]]
    vv_ref["fixed logfc"] = -vv_ref["logfc"].astype(float)
    vv_alt["fixed logfc"] = vv_alt["logfc"].astype(float)
    vv = pd.concat([vv_ref, vv_alt])
except:
    pass

te_blacklist = pd.read_csv("BlacklistTE_Variants.txt", sep = "\t")
    
v = v[~v["Position"].isin(te_blacklist["Position"])]
vv = vv[~vv["Position"].isin(te_blacklist["Position"])]



In [ ]:
#Read in the CAE data
v_ee = pd.read_csv("Taus_HumanDerived_Group1_WithNeuron.txt", sep = "\t").set_index("Position")
vv_ee = pd.read_csv("Taus_Polymorphic_Group1_WithNeuron.txt", sep = "\t").set_index("Position")

v.index = v["Position"]
v = v.join(v_ee)
vv.index = vv["Position"]
vv = vv.join(vv_ee)



In [ ]:
#Hard coded filter for fetal cortical excitatory neurons
v2 = v[v["abs logfc"] > 0.1034199]
vv2 = vv[vv["abs logfc"] > 0.1034199]

In [ ]:
#Make plot for absolute log fold-change
vvv = prepare_alpha(v2, vv2, stat = "abs logfc")

cuttt = 0.3

z = list(vv2["Tau_abs_logfc"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

alpha = compute_alpha_cutoff(vvv, plot = True, window = [0, 1], cutoff = cutoff, title = "Using absolute log fold-change")
plt.xlabel("Predicted absolute log$_{2}$ fold-change in CA", size = 20)
plt.show()
print("Tau_abs_logfc alpha", alpha)
print("Tau_abs_logfc p-value", (fisher_exact(alpha[-2])[1] + fisher_exact(alpha[-3])[1])/2)

In [ ]:
#Getting list of genes to show on Biorender plot
vk = v[(v["Tau_abs_logfc"] > 0.95) & (v["abs logfc"] > 0.5)]
syngo = pd.read_csv("../Cell_Type_Prop/Cell_Type_Prop/syngo_genes.csv")
syngo = syngo["hgnc_symbol"]

vk_syn = vk[vk["NearestGene"].isin(syn[syn["go_name"] == "postsynaptic density (GO:0014069)"]["hgnc_symbol"])]
for i in np.unique(vk_syn["NearestGene"]):
    print(i)

In [ ]:
#Preparing to plot the CAE stuff, can change cell type to ITL23
v.index = v["Position"]
vv.index = vv["Position"]

dl_path = "./"
dl_prefix = "LiangSteinNeuron"
cts_metric = "EE"
cts_fixed = pd.read_csv(dl_path + dl_prefix + "_EEs_HumanDerived_WithNeuron.txt.gz", sep = "\t").set_index("Position")
cts_poly = pd.read_csv(dl_path + dl_prefix + "_EEs_Polymorphic_WithNeuron.txt.gz", sep = "\t").set_index("Position")
cts_fixed = cts_fixed[["EE_Allele1", "EE_Allele2"]]
cts_poly = cts_poly[["EE_Allele1", "EE_Allele2"]]

v = v.join(cts_fixed).dropna().drop_duplicates("Position")
vv = vv.join(cts_poly).dropna().drop_duplicates("Position")
print(v)
print(vv)

v.index = v["Position"]
vv.index = vv["Position"]

v["EE_Allele1"] = v["EE_Allele1"].astype(float)
v["EE_Allele2"] = v["EE_Allele2"].astype(float)
vv["EE_Allele1"] = vv["EE_Allele1"].astype(float)
vv["EE_Allele2"] = vv["EE_Allele2"].astype(float)

v["EE_Dif"] = v["EE_Allele1"] - v["EE_Allele2"]
vv["EE_Dif"] = vv["EE_Allele1"] - vv["EE_Allele2"]

print(v.shape[0], vv.shape[0])

v["logfc"] = v["logfc"].astype(float)
v["abs logfc"] = v["abs logfc"].astype(float)

vv["logfc"] = vv["logfc"].astype(float)
vv["fixed logfc"] = vv["fixed logfc"].astype(float)
vv["abs logfc"] = vv["abs logfc"].astype(float)

signs = []
for index, row in vv.iterrows():
    if np.sign(row["logfc"]) == np.sign(row["fixed logfc"]):
        signs.append(1)
    else:
        signs.append(-1)
vv["SignFlip"] = signs
vv["Fixed EE_Dif"] = vv["SignFlip"]*vv["EE_Dif"]
v["Fixed EE_Dif"] = v["EE_Dif"]

v = v.drop_duplicates("Position")
vv = vv.drop_duplicates("Position")
v

In [ ]:
v2 = v[np.abs(v["EE_Dif"]) > 0.025].copy()
v2

In [ ]:
#Plotting using cell type-specifity
vvv = prepare_alpha(v2, vv2, stat = "Tau_abs_logfc")

cuttt = 0.3

z = list(vv2["Tau_abs_logfc"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

alpha = compute_alpha_cutoff(vvv, plot = True, window = [0, 1], cutoff = cutoff, title = "Using cell type-specificity")
plt.show()
print("Tau_abs_logfc alpha", alpha)
print("Tau_abs_logfc p-value", (fisher_exact(alpha[-2])[1] + fisher_exact(alpha[-3])[1])/2)

In [ ]:
print("Tau_abs_logfc alpha", alpha)
print("Tau_abs_logfc p-value", (fisher_exact(alpha[-2])[1] + fisher_exact(alpha[-3])[1])/2)

In [ ]:
#Now for absolute log fold-change
vvv = prepare_alpha(v2, vv2, stat = "abs logfc")

cuttt = 0.6

z = list(vv2["abs logfc"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

alpha = compute_alpha_cutoff(vvv, plot = True, window = [0, 1], cutoff = cutoff, title = "Using absolute log fold-change")
plt.show()
print("abs_logfc alpha", alpha)
print("abs_logfc p-value", (fisher_exact(alpha[-2])[1] + fisher_exact(alpha[-3])[1])/2)

In [ ]:
### Create inputs to GREAT to test whether sites with large abs logfc are enriched near certain types of genes

#Change 0.1 to 0.25 for NFIB and other TFs with large effects on CA
v2_cts_fore = v2_cts.copy()

v2_cts_fore["Chrom"] = [x.split(":")[0] for x in v2_cts_fore["Position"]]
v2_cts_fore["Pos1"] = [int(x.split(":")[1]) for x in v2_cts_fore["Position"]]
v2_cts_fore["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2_cts_fore["Position"]]
v2_cts_fore = v2_cts_fore[["Chrom", "Pos1", "Pos2"]]

#Need to change 
v2_cts_fore.to_csv("LSN_CTS_AbsG0.1_Fixed_Near.bed", sep = "\t", header = None, index = None)

#v2_cts["Chrom"] = [x.split(":")[0] for x in v2_cts["Position"]]
#v2_cts["Pos1"] = [int(x.split(":")[1]) for x in v2_cts["Position"]]
#v2_cts["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2_cts["Position"]]

#Set to 0.05 for MEF2A
#Change 0.025 to 0.1 for NFIB and other TFs with large effects on CA
v2_cts_back = pd.concat([v2_cts, v2_ncts])
v2_cts_back["Chrom"] = [x.split(":")[0] for x in v2_cts_back["Position"]]
v2_cts_back["Pos1"] = [int(x.split(":")[1]) for x in v2_cts_back["Position"]]
v2_cts_back["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2_cts_back["Position"]]
v2_cts_back = v2_cts_back[["Chrom", "Pos1", "Pos2"]]
v2_cts_back.to_csv("LSN_NCTS_AbsG0.1_Fixed_Background_Near.bed", sep = "\t", header = None, index = None)

#Not used, making the background polymorphisms (which are much rarer) does not work well
#vv2_cts["Chrom"] = [x.split(":")[0] for x in vv2_cts["Position"]]
#vv2_cts["Pos1"] = [int(x.split(":")[1]) for x in vv2_cts["Position"]]
#vv2_cts["Pos2"] = [int(x.split(":")[1]) + 1 for x in vv2_cts["Position"]]
#v2_cts_back = pd.concat([v2_cts_fore, vv2_cts])
#v2_cts_back = v2_cts_back[["Chrom", "Pos1", "Pos2"]]
#v2_cts_back.to_csv("NR1D1_AbsG0.1_Fixed_Background_Poly_ITL23_FilterNewTEs.bed", sep = "\t", header = None, index = None)

In [ ]:
#Permutation method

#Top enrichments by great are mostly synaptic/neuron-related for LSN
#Top gene is VWC2L, which has highly specific downregulation in motor neurons

#Second from top gene for preGC is FGF10 which may play a key role in the process: https://pubmed.ncbi.nlm.nih.gov/20876224/
#Maybe more surprisingly, synaptic terms are also enriched for the preGC cells
#Though again cell type-specificity depends on the other cell types, so this makes some sense

#v2p = v2[v2["NearestGene"].isin(["VWC2L"])]
v2p = v2[v2["NearestGene"].isin(["FGF10"])]
#syngo = pd.read_csv("../Cell_Type_Prop/Cell_Type_Prop/syngo_genes.csv")
#keep = syngo["hgnc_symbol"]
#v2p = v2[v2["NearestGene"].isin(keep)]
real = np.mean(v2p["Tau_abs_logfc"])
size = v2p.shape[0]

#Only doing 1000 random samples for synaptic genes
fakes = []
for i in range(10000):
    np.random.seed(i)
    fake = np.mean(v2.sample(n = size, replace = False)["Tau_abs_logfc"])
    fakes.append(fake)
print(norm.sf((real - np.mean(fakes))/np.std(fakes)))

sns.kdeplot(fakes, fill = True, color = "#40A94D")
plt.vlines(x = real, ymin = 0, ymax = 60, color = "#F42FF5")
plt.ylim(0, 60)
plt.xlabel("Mean tau for abs logfc", size = 14)
plt.ylabel("Density of random subsets", size = 14)
plt.title("CA changes near $\it{FGF10}$ are more cell type-specific", size = 14)

In [ ]:
sns.kdeplot(fakes, fill = True, color = "#40A94D")
plt.vlines(x = real, ymin = 0, ymax = 60, color = "#F42FF5")
plt.ylim(0, 60)
plt.xlabel("Mean tau for abs logfc", size = 14)
plt.ylabel("Density of random subsets", size = 14)
plt.title("CA changes near VWC2L are more cell type-specific", size = 14)

In [ ]:
#Top enrichments by great are mostly synaptic/neuron-related for LSN
#Top gene is VWC2L, which has highly specific downregulation in motor neurons

#Second from top gene for preGC is FGF10 which may play a key role in the process: https://pubmed.ncbi.nlm.nih.gov/20876224/
#Maybe more surprisingly, synaptic terms are also enriched for the preGC cells
#Though again cell type-specificity depends on the other cell types, so this makes some sense

#v2p = v2[v2["NearestGene"].isin(["VWC2L"])]
#v2p = v2[v2["NearestGene"].isin(["FGF10"])]
syngo = pd.read_csv("../Cell_Type_Prop/Cell_Type_Prop/syngo_genes.csv")
keep = syngo["hgnc_symbol"]
v2p = v2[v2["NearestGene"].isin(keep)]
real = np.mean(v2p["Tau_abs_logfc"])
size = v2p.shape[0]

#Only doing 1000 random samples for synaptic genes
fakes = []
for i in range(10000):
    np.random.seed(i)
    fake = np.mean(v2.sample(n = size, replace = False)["Tau_abs_logfc"])
    fakes.append(fake)
print(norm.sf((real - np.mean(fakes))/np.std(fakes)))

sns.kdeplot(fakes, fill = True, color = "#40A94D")
plt.vlines(x = real, ymin = 0, ymax = 60, color = "#F42FF5")
plt.ylim(0, 60)
plt.xlabel("Mean tau for abs logfc", size = 14)
plt.ylabel("Density of random subsets", size = 14)
plt.title("CA changes near synaptic genes are more cell type-specific", size = 14)

In [ ]:
#Repeating for absolute log fold-change

#v2p = v2[v2["NearestGene"].isin(["VWC2L"])]
#v2p = v2[v2["NearestGene"].isin(["FGF10"])]
syngo = pd.read_csv("../Cell_Type_Prop/Cell_Type_Prop/syngo_genes.csv")
keep = syngo["hgnc_symbol"]
v22 = v2.copy()
v2p = v22[v22["NearestGene"].isin(keep)]
real = np.mean(v2p["abs logfc"])
size = v2p.shape[0]

#Only doing 1000 random samples for synaptic genes
fakes = []
for i in range(1000):
    np.random.seed(i)
    fake = np.mean(v22.sample(n = size, replace = False)["abs logfc"])
    fakes.append(fake)
print(norm.sf((real - np.mean(fakes))/np.std(fakes)))

#sns.kdeplot(fakes, fill = True, color = "#40A94D")
#plt.vlines(x = real, ymin = 0, ymax = 60, color = "#F42FF5")
#plt.ylim(0, 60)
#plt.xlabel("Mean tau for abs logfc", size = 14)
#plt.ylabel("Density of random subsets", size = 14)
#plt.title("CA changes near synaptic genes are more cell type-specific", size = 14)

In [ ]:
sns.kdeplot(fakes, fill = True, color = "#40A94D")
plt.vlines(x = real, ymin = 0, ymax = 1200, color = "#F42FF5")
plt.ylim(0, 1200)
plt.xlabel("Mean dTau", size = 16)
plt.ylabel("Density of random subsets", size = 16)
plt.title("Synaptic genes", size = 18)
plt.xticks(size = 10)
plt.yticks(size = 10)

In [ ]:
#Plotting GREAT enrichments for preGranulosa
k = []
for index, row in dff.iterrows():
    if row["HyperFdrQ"] < 0.05:
        k.append("FDR < 0.05")
    else:
        k.append("Not significant")
mult = 1.3


dff['Significance'] = k
fig, ax = plt.subplots(figsize = (9, 6))
sns.scatterplot(data = dff, x = "RegionFoldEnrich", y = "-Log$_{10}$(FDR)", hue = "Significance", palette = {"FDR < 0.05":"red", "Not significant":"grey"})
plt.title("pre-Granulosa cells", size = 16*mult)
plt.ylabel("-Log$_{10}$(FDR)", size = 14*mult)
plt.xlabel("Fold enrichment", size = 14*mult)
plt.xticks(size = 12*mult)
plt.yticks(size = 12*mult)
plt.legend(fontsize = 12*mult)